## Process Colocate csv

In [2]:
import pandas as pd
import random
pd.set_option("display.max_columns", None)


single = pd.read_csv('./single_metric2.csv', index_col=0)
colo = pd.read_csv('./colocate/colocate_metric.csv', index_col=0)
#NOTE
single = single.drop(columns=['time'])

single.reset_index(drop=True, inplace=True)
colo.reset_index(drop=True, inplace=True)

# Uniform Dataset and Models
single.replace(
['imagenet', 'cifar10', 'shapenet', 'LSUN', 'LunarLander-v2', 'BipedalWalker-v3', 'MovieLens ', 'Wikitext2', 'SQUAD', 'multi30k'],
['ImageNet', 'CIFAR-10', 'ShapeNet', 'LSUN', 'LunarLander', 'BipedalWalker', 'MovieLens', 'Wikitext2', 'SQuAD', 'Multi30k'], inplace=True)

single.replace(
['resnet50', 'mobilenet_v3_small', 'EfficientNetB0', 'pointnet', 'dcgan', 'NeuMF-pre', 'bert', 'transformer'],
['ResNet50', 'MobileNetV3', 'EfficientNet', 'PointNet', 'DCGAN', 'NeuMF', 'BERT', 'Transformer'], inplace=True)

colo.replace(
['EfficientNetB0','NeuMF-pre'],
['EfficientNet', 'NeuMF'], inplace=True)

# single

In [3]:
def query_speed(single, m, d, b, a, g):
    info = single.query("model == @m and batchsize == @b and dataset == @d and amp == @a and gpu_num == @g")
    if not info.empty:
        return info["speed"].values[0]
    else:
        print(m, d, b, a, g)


for i in range(len(colo)):
    # print(i)
    speed1 = query_speed(single, colo.at[i,'model1'], colo.at[i,'dataset1'], colo.at[i,'batchsize1'], colo.at[i,'amp1'], colo.at[i,'gpu_num1'])
    speed2 = query_speed(single, colo.at[i,'model2'], colo.at[i,'dataset2'], colo.at[i,'batchsize2'], colo.at[i,'amp2'], colo.at[i,'gpu_num2'])
    colo.at[i,'speed1'] = round(colo.at[i,'speed1'] / speed1, 3)
    colo.at[i,'speed2'] = round(colo.at[i,'speed2'] / speed2, 3)
# colo.to_csv('colocate_info.csv')

In [57]:
colo.to_csv('colocate_info.csv')

Bad performance result

In [ ]:
bad = colo[(colo['speed1'] < 0.8) & (colo['speed2'] < 0.8)].copy()
bad.drop(bad[(bad['model1']=='PPO') | (bad['model1']=='TD3') | (bad['model2']=='PPO') | (bad['model2']=='TD3')].index, inplace=True)
# bad.to_csv('bad_colo.csv')
bad

Batchsize influence

In [128]:
colo_nml = pd.read_csv('colocate_info.csv')
colo_nml.drop(colo_nml[(colo_nml['model1']=='PPO') | (colo_nml['model1']=='TD3') | (colo_nml['model2']=='PPO') | (colo_nml['model2']=='TD3')].index, inplace=True)
colo_amp0 = colo_nml[(colo_nml['amp1']==0) & (colo_nml['amp2']==0)]

print(f'Batchsize 32')
print(colo_amp0[(colo_amp0['batchsize1']==32) & (colo_amp0['batchsize2']==32)]['speed1'].mean())
print(colo_amp0[(colo_amp0['batchsize1']==32) & (colo_amp0['batchsize2']==32)]['speed2'].mean())

print(f'Batchsize 64')
print(colo_amp0[(colo_amp0['batchsize1']==64) & (colo_amp0['batchsize2']==64)]['speed1'].mean())
print(colo_amp0[(colo_amp0['batchsize1']==64) & (colo_amp0['batchsize2']==64)]['speed2'].mean())
print(f'Batchsize 128')
print(colo_amp0[(colo_amp0['batchsize1']==128) & (colo_amp0['batchsize2']==128)]['speed1'].mean())
print(colo_amp0[(colo_amp0['batchsize1']==128) & (colo_amp0['batchsize2']==128)]['speed2'].mean())


Batchsize 32
0.9683928571428574
0.96525
Batchsize 64
0.8641891891891894
0.8994864864864869
Batchsize 128
0.7430540540540539
0.8225135135135136


Amp influence

In [68]:
def amp_speed(bs):
    colo_bs32 = colo_nml[(colo_nml['batchsize1']==bs) & (colo_nml['batchsize2']==bs)]
    s1a0 = colo_bs32[(colo_bs32['amp1']==0) & (colo_bs32['amp2']==0)]['speed1'].mean()
    s2a0 = colo_bs32[(colo_bs32['amp1']==0) & (colo_bs32['amp2']==0)]['speed2'].mean()

    s1a1 = colo_bs32[(colo_bs32['amp1']==1) & (colo_bs32['amp2']==1)]['speed1'].mean()
    s2a1 = colo_bs32[(colo_bs32['amp1']==1) & (colo_bs32['amp2']==1)]['speed2'].mean()
    return s1a0, s2a0, s1a1, s2a1


colo_nml = pd.read_csv('colocate_info.csv')
colo_nml.drop(colo_nml[(colo_nml['model1']=='PPO') | (colo_nml['model1']=='TD3') | (colo_nml['model2']=='PPO') | (colo_nml['model2']=='TD3')].index, inplace=True)

mean = []
for bs in [32, 64, 128]:
    s1a0, s2a0, s1a1, s2a1 = amp_speed(bs)
    mean1 = {'batchsize': bs, 'amp': 0, 'avg_s1': s1a0, 'avg_s2': s2a0, 'mean': (s1a0+s2a0)/2}
    mean2 = {'batchsize': bs, 'amp': 1, 'avg_s1': s1a1, 'avg_s2': s2a1, 'mean': (s1a1+s2a1)/2}
    mean.append(mean1)
    mean.append(mean2)

pd.DataFrame(mean)


,batchsize,amp,avg_s1,avg_s2,mean
0,32,0,0.968393,0.965250,0.966821
1,32,1,1.014857,0.988679,1.001768
2,64,0,0.864189,0.899486,0.881838
3,64,1,0.968162,0.962081,0.965122
4,128,0,0.743054,0.822514,0.782784
5,128,1,0.884054,0.937811,0.910932


Takeaway

small model: 'mobilenet_v3_small', 'EfficientNetB0', 'pointnet', 'mobilenetv2'

medium model: 'resnet18', 'resnet50', 'vgg11', 'dcgan', 'LSTM', 'NeuMF'

large model: 'bert', 'Transformer',


In [14]:
colo_nml = pd.read_csv('colocate_info.csv')
small = colo_nml[(colo_nml['model1']=='ResNet18') & (colo_nml['model2']=='DCGAN') & (colo_nml['batchsize1']==64) & (colo_nml['batchsize2']==32) & (colo_nml['amp1']==0) & (colo_nml['amp2']==1)]

# small[(small['speed2']<0.8) & (small['speed1']<0.8)]
small


,Unnamed: 0,model1,dataset1,gpu_num1,batchsize1,amp1,speed1,model2,dataset2,gpu_num2,batchsize2,amp2,speed2,gpu_util,gmem_util,gmem,added_util
1034,205,ResNet18,CIFAR-10,1,64,0,0.926,DCGAN,LSUN,1,32,1,0.9,92.579,56.711,5412,116.464


 Self Colocate

In [143]:
same = []
for i in range(len(colo)):
    m1, m2 = colo.at[i, 'model1'], colo.at[i, 'model2']
    a1, a2 = colo.at[i, 'amp1'], colo.at[i, 'amp2']
    b1, b2 = colo.at[i, 'batchsize1'], colo.at[i, 'batchsize2']
    if m1 == m2 and a1 == a2 and b1 == b2:
        same.append(i)
samedf = colo.iloc[same]
samedf.to_csv('colocate_same2.csv', index=False)

In [5]:
colo_nml = pd.read_csv('colocate_same.csv')
small = colo_nml[(colo_nml['batchsize1']==64) & (colo_nml['batchsize2']==64) & (colo_nml['amp1']==0) & (colo_nml['amp2']==0)]

# small[(small['speed2']<0.8) & (small['speed1']<0.8)]
small

,model1,dataset1,gpu_num1,batchsize1,amp1,speed1,model2,dataset2,gpu_num2,batchsize2,amp2,speed2,gpu_util,gmem_util,gmem
2,PointNet,ShapeNet,1,64,0,0.973,PointNet,ShapeNet,1,64,0,0.971,16.761,14.627,13496
6,LSTM,Wikitext2,1,64,0,0.563,LSTM,Wikitext2,1,64,0,0.563,100.000,84.286,7790
10,NeuMF,MovieLens,1,64,0,0.980,NeuMF,MovieLens,1,64,0,0.986,20.724,4.816,4122
15,TD3,BipedalWalker,1,64,0,1.000,TD3,BipedalWalker,1,64,0,1.000,23.439,0.712,4090
19,ResNet18,CIFAR-10,1,64,0,0.648,ResNet18,CIFAR-10,1,64,0,0.652,98.658,68.513,5342
25,MobileNetV2,CIFAR-10,1,64,0,0.829,MobileNetV2,CIFAR-10,1,64,0,0.846,97.851,64.081,6076
31,EfficientNet,CIFAR-10,1,64,0,0.874,EfficientNet,CIFAR-10,1,64,0,0.827,51.625,13.083,5554
37,VGG,CIFAR-10,1,64,0,0.861,VGG,CIFAR-10,1,64,0,0.873,87.027,59.253,7190
42,PPO,LunarLander,1,64,0,1.000,PPO,LunarLander,1,64,0,1.000,22.818,0.026,4074
46,MobileNetV3,ImageNet,1,64,0,0.736,MobileNetV3,ImageNet,1,64,0,0.739,100.000,66.625,6554


Save cumulated gpu_util

In [8]:
colo = pd.read_csv('colocate_info.csv', index_col=0)
single = pd.read_csv('./single_mean.csv')
def query_util(single, m, d, b, a, g):
    info = single.query("model == @m and batchsize == @b and dataset == @d and amp == @a and gpu_num == @g")
    if not info.empty:
        return info["gpu_util"].values[0]
    else:
        print(m, d, b, a, g)


for i in range(len(colo)):
    # print(i)
    util1 = query_util(single, colo.at[i,'model1'], colo.at[i,'dataset1'], colo.at[i,'batchsize1'], colo.at[i,'amp1'], colo.at[i,'gpu_num1'])
    util2 = query_util(single, colo.at[i,'model2'], colo.at[i,'dataset2'], colo.at[i,'batchsize2'], colo.at[i,'amp2'], colo.at[i,'gpu_num2'])
    colo.at[i,'added_util'] = util1 + util2
# colo.to_csv('colocate_info.csv')       